In [1]:
# LSTM for international airline passengers problem with regression framing
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
df = read_csv('predict/weatherKZZM.csv', usecols=[0, 3, 4, 5, 6,7,8], engine='python') 

import warnings
print(df.head())
warnings.filterwarnings("ignore")

2024-10-01 20:02:00.312709: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-01 20:02:00.364767: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-01 20:02:00.365556: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-01 20:02:01.319288: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


       Date  Humidity1  Humidity2  Max_Temp  Min_Temp  carbon1  carbon2
0  1/1/2024         82         53      30.2      14.4      347      354
1  1/2/2024         82         60      30.0      14.2      336      376
2  1/3/2024         82         60      30.2      14.5      348      402
3  1/4/2024         82         72      30.2      14.4      384      423
4  1/5/2024         82         60      29.6      14.8      398      450


In [2]:
df.Date =  pd.to_datetime(df.Date)
df = df.set_index("Date")

In [3]:
train,test = df[:-1],df[:-1]
scaler = MinMaxScaler()
scaler.fit(train)
train =scaler.transform(train)
test = scaler.transform(test)
print(df[-1:])

            Humidity1  Humidity2  Max_Temp  Min_Temp  carbon1  carbon2
Date                                                                  
2024-07-31         77         79      30.2      27.3      431      578


In [ ]:
n_input = 7
n_features = 6
generator = TimeseriesGenerator(train,train,length=n_input,batch_size=1000)#6
model = Sequential()
model.add(LSTM(100,activation="relu",input_shape=(n_input,n_features)))
model.add(Dropout(0.7));
model.add(Dense(6))
model.compile(optimizer="adam",loss="mse")
model.fit(generator, epochs=3000)

Epoch 1/3000
1/1 [==============================] - 1s 1s/step - loss: 0.3705
Epoch 2/3000
1/1 [==============================] - 0s 34ms/step - loss: 0.3506
Epoch 3/3000
1/1 [==============================] - 0s 32ms/step - loss: 0.3293
Epoch 4/3000
1/1 [==============================] - 0s 30ms/step - loss: 0.3102
Epoch 5/3000
1/1 [==============================] - 0s 33ms/step - loss: 0.2911
Epoch 6/3000
1/1 [==============================] - 0s 36ms/step - loss: 0.2791
Epoch 7/3000
1/1 [==============================] - 0s 30ms/step - loss: 0.2545
Epoch 8/3000
1/1 [==============================] - 0s 33ms/step - loss: 0.2440
Epoch 9/3000
1/1 [==============================] - 0s 33ms/step - loss: 0.2230
Epoch 10/3000
1/1 [==============================] - 0s 34ms/step - loss: 0.2068
Epoch 11/3000
1/1 [==============================] - 0s 34ms/step - loss: 0.1924
Epoch 12/3000
1/1 [==============================] - 0s 32ms/step - loss: 0.1797
Epoch 13/3000
1/1 [====================

In [ ]:
pred_list = []
import numpy as np
batch = train[-n_input:].reshape((1,n_input,n_features))
for i in range(n_input):
    pred_list.append(model.predict(batch)[0])
    batch = np.append(batch[:,1:,:],[[pred_list[i]]],axis=1)

In [ ]:
df_predict = pd.DataFrame(scaler.inverse_transform(pred_list),index=df[-n_input:].index,columns=["Hum1Pred","Hum2Pred","MaxTempPred","MinTempPred","carbon1Predict","carbon2Predict"])

In [ ]:
df_test = pd.concat([df,df_predict],axis=1)
df_test

In [ ]:
plt.figure(figsize=(20, 15))
plt.plot(df_test.index, df_test["Min_Temp"], label="Actual", color='g', marker='o')
plt.plot(df_test.index, df_test["MinTempPred"], label="Predict", color='r', marker='X')
plt.title('Temperature Day Time', fontsize=20)
plt.xlabel('Date', fontsize=15)
plt.ylabel('Temperature (°C)', fontsize=15)
min_temp = df_test["Min_Temp"].min() // 10 * 10  # Round down to nearest 10
max_temp = df_test["Min_Temp"].max() // 10 * 10 + 10  # Round up to nearest 10
plt.yticks(range(int(min_temp), int(max_temp) + 1, 10))  # Create ticks from min to max with step of 10
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(20,15))
plt.plot(df_test.index,df_test["Max_Temp"], label="Actual",color='g',marker='o')
plt.plot(df_test.index,df_test["MaxTempPred"], label="Predict",color='r',marker='X')
plt.title(' Temperature Night Time', fontsize=20)
plt.xlabel('Date', fontsize=15)
plt.ylabel('Temperature (°C)', fontsize=15)
min_temp = df_test["Max_Temp"].min() // 10 * 10  # Round down to nearest 10
max_temp = df_test["Max_Temp"].max() // 10 * 10 + 10  # Round up to nearest 10
plt.yticks(range(int(min_temp), int(max_temp) + 1, 10))  # Create ticks from min to max with step of 10
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(20,15))
plt.plot(df_test.index,df_test["Humidity1"], label="Actual",color='g',marker='o')
plt.plot(df_test.index,df_test["Hum1Pred"], label="Predict",color='r',marker='X')
plt.title(' Humidity Day Time', fontsize=20)
plt.xlabel('Date', fontsize=15)
plt.ylabel('Humidity', fontsize=15)
min_humi = df_test["Humidity1"].min() // 10 * 10  # Round down to nearest 10
max_humi = df_test["Humidity1"].max() // 10 * 10 + 10  # Round up to nearest 10
plt.yticks(range(int(min_humi), int(max_humi) + 1, 10))  # Create ticks from min to max with step of 10
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(20,15))
plt.plot(df_test.index,df_test["Humidity2"],color='g',marker='o')
plt.plot(df_test.index,df_test["Hum2Pred"],color='r',marker='X')
plt.title(' Humidity Night Time', fontsize=20)
plt.xlabel('Date', fontsize=15)
plt.ylabel('Humidity', fontsize=15)
min_humi = df_test["Humidity2"].min() // 10 * 10  # Round down to nearest 10
max_humi = df_test["Humidity2"].max() // 10 * 10 + 10  # Round up to nearest 10
plt.yticks(range(int(min_humi), int(max_humi) + 1, 10))  # Create ticks from min to max with step of 10
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(20,15))
plt.plot(df_test.index,df_test["carbon1"],color='g',marker='o')
plt.plot(df_test.index,df_test["carbon1Predict"],color='r',marker='X')
plt.legend()
min_carbon = df_test["carbon1"].min() // 10 * 10  # Round down to nearest 10
max_carbon = df_test["carbon1"].max() // 10 * 10 + 10  # Round up to nearest 10
plt.yticks(range(int(min_carbon), int(max_carbon) + 1, 10))  # Create ticks from min to max with step of 10
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(20,15))
plt.plot(df_test.index,df_test["carbon2"],color='g',marker='o')
plt.plot(df_test.index,df_test["carbon2Predict"],color='r',marker='X')
min_carbon = df_test["carbon2"].min() // 10 * 10  # Round down to nearest 10
max_carbon = df_test["carbon2"].max() // 10 * 10 + 10  # Round up to nearest 10
plt.yticks(range(int(min_carbon), int(max_carbon) + 1, 10))  # Create ticks from min to max with step of 10
plt.legend()
plt.show()